In [31]:
import pandas as pd
X_train = pd.read_csv('D:/train_fix.csv')
Y_train = pd.read_csv('D:/train_label.csv')
X_test = pd.read_csv('D:/test.csv')
Y_test = pd.read_csv('D:/test_nolabel.csv')

In [32]:
X_train

,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,Resort Hotel,0,342,2015,July,27,1,0,0,...,No Deposit,NaN,NaN,0,Transient,-6.305161,0,0,Check-Out,7/1/2015
1,1,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6.0,NaN,0,Transient,75.052227,0,0,Check-Out,7/3/2015
2,2,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6.0,NaN,0,Transient,74.546401,0,0,Check-Out,7/3/2015
3,3,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6.0,NaN,0,Transient,76.376288,0,0,Check-Out,7/3/2015
4,4,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6.0,NaN,0,Transient,49.411647,0,0,Check-Out,7/3/2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91525,91526,Resort Hotel,1,19,2017,March,13,31,0,2,...,No Deposit,250.0,NaN,0,Transient,79.223571,0,1,Canceled,3/22/2017
91526,91527,Resort Hotel,0,28,2017,March,13,31,0,2,...,No Deposit,NaN,NaN,0,Transient,-6.822102,0,0,Check-Out,4/2/2017
91527,91528,Resort Hotel,0,2,2017,March,13,31,0,1,...,No Deposit,NaN,NaN,0,Transient,90.814554,0,2,Check-Out,4/1/2017
91528,91529,Resort Hotel,0,30,2017,March,13,31,3,7,...,No Deposit,250.0,NaN,0,Transient,38.135565,0,1,Check-Out,4/10/2017


In [41]:
X_train_cdf['hotel_City Hotel']

0        0
1        1
2        1
3        1
4        1
        ..
91525    0
91526    0
91527    0
91528    0
91529    0
Name: hotel_City Hotel, Length: 91530, dtype: uint8

In [29]:
np.array(X_train['country']=='PRT').dot([1 for i in range(91531)])/91531 *100


44.07250002731315

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [33]:
# data processing for is_canceled prediction
def data_process_cancel(X):
    
    # drop the columns only in train data
    only_inTrain = ['reservation_status', 'reservation_status_date']
    X = X.loc[:, ~X.columns.isin(only_inTrain)].copy() 
    
    # drop the columns not related to is_canceled
    not_related = ['ID', 'days_in_waiting_list', 'required_car_parking_spaces',
                  'reserved_room_type', 'assigned_room_type']
    X = X.loc[:, ~X.columns.isin(not_related)]
    
    # missing data
    X['children'] = X['children'].fillna(0)
    X['country'] = X['country'].fillna(X['country'].value_counts().iloc[0]) # fill in the max
    X['company'] = (X['company'].notnull()).astype('int') # replace all non-NaN entries of a dataframe with 1 and all NaN with 0
    
    kagent = list(X_train['agent'].value_counts().iloc[:20].index)
    X['agent'] = np.where(X['agent'].isin(kagent), X['agent'], 0)
    X['agent'] = X['agent'].fillna(0)
    
    # combine
    X['child'] = X['children'] + X['babies']
    X = X.drop(['children', 'babies'], axis=1)
    
    # add new column
    X['previous_cancel_rate'] =  X['previous_cancellations']/(X['previous_cancellations']+X['previous_bookings_not_canceled'])
    X['previous_cancel_rate'] = X['previous_cancel_rate'].fillna(0)
    X = X.drop(['previous_cancellations', 'previous_bookings_not_canceled'], axis=1)  
    
    # threshold
    X['lead_time'] = np.where(X['lead_time']>400, 1, 0)
    
    # get dummy
    dummy_cols = ['hotel', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'meal', 'country', 'market_segment', 'distribution_channel',
       'deposit_type', 'company', 'customer_type', 'agent']
    X = pd.get_dummies(X, columns=dummy_cols)
    return X

X_train_cdf = data_process_cancel(X_train.copy())

In [18]:
def test_process_cancel(X):
    
    # drop the columns not related to is_canceled
    not_related = ['ID', 'days_in_waiting_list', 'required_car_parking_spaces',
                  'reserved_room_type', 'assigned_room_type']
    X = X.loc[:, ~X.columns.isin(not_related)]
    
    # missing data
    X['country'] = X['country'].fillna(X['country'].value_counts().iloc[0]) # fill in the max
    X['company'] = (X['company'].notnull()).astype('int') # replace all non-NaN entries of a dataframe with 1 and all NaN with 0
    
    kagent = list(X_train['agent'].value_counts().iloc[:20].index)
    X['agent'] = np.where(X['agent'].isin(kagent), X['agent'], 0)
    X['agent'] = X['agent'].fillna(0)
    
    # combine
    X['child'] = X['children'] + X['babies']
    X = X.drop(['children', 'babies'], axis=1)
    
    # add new column
    X['previous_cancel_rate'] =  X['previous_cancellations']/(X['previous_cancellations']+X['previous_bookings_not_canceled'])
    X['previous_cancel_rate'] = X['previous_cancel_rate'].fillna(0)
    X = X.drop(['previous_cancellations', 'previous_bookings_not_canceled'], axis=1)  
    
    # threshold
    X['lead_time'] = np.where(X['lead_time']>400, 1, 0)
    
    # get dummy
    dummy_cols = ['hotel', 'arrival_date_year', 'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'meal', 'country', 'market_segment', 'distribution_channel',
       'deposit_type', 'company', 'customer_type', 'agent']
    X = pd.get_dummies(X, columns=dummy_cols)
    

    T = X_train_cdf.columns.tolist()
    Mi = [i for i in X_train_cdf if i not in X]
    for i in Mi:
        X[i] = 0

    X=X[X_train_cdf.columns.tolist()]    
    
    
    return X

X_test_cdf = test_process_cancel(X_test.copy())

C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\USER\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

In [ ]:
# train data
X_train_c = X_train_cdf.to_numpy()
Y_train_c = X_train['is_canceled'].copy().to_numpy()

# test data
X_test_c = X_test_cdf.to_numpy()

In [31]:
# All data
X_train_c = X_train_cdf.loc[:, ~X_train_cdf.columns.isin(['is_canceled','adr'])].to_numpy()
Y_train_Cancel = X_train_cdf['is_canceled'].copy().to_numpy()
Y_train_ADR = X_train_cdf['adr'].copy().to_numpy()

# Shuffle data for train
Train_copy = X_train_cdf.sample(frac=1, random_state=1) # shuffle

Tra_ADR = Train_copy['adr']
Tra_X = Train_copy.drop(['is_canceled','adr'], axis=1)
Tra_Y = Train_copy['is_canceled']

### Train & Validation

In [59]:
# 5 folds cv
Tra_5folds_ADR = [Tra_ADR[18306:],Tra_ADR[0:18306].append(Tra_ADR[36612:]),Tra_ADR[0:36612].append(Tra_ADR[54918:]),
                  Tra_ADR[0:54918].append(Tra_ADR[73224:]),Tra_ADR[0:73224]]
Tra_5folds_X = [Tra_X.iloc[18306:],Tra_X.iloc[0:18306].append(Tra_X.iloc[36612:]),Tra_X.iloc[0:36612].append(Tra_X.iloc[54918:]),
                  Tra_X.iloc[0:54918].append(Tra_X.iloc[73224:]),Tra_X.iloc[0:73224]]
Tra_5folds_Y = [Tra_Y[18306:],Tra_Y[0:18306].append(Tra_Y[36612:]),Tra_Y[0:36612].append(Tra_Y[54918:]),
                  Tra_Y[0:54918].append(Tra_Y[73224:]),Tra_Y[0:73224]]

Val_5folds_X = [Tra_X.iloc[0:18306],Tra_X.iloc[18306:36612],Tra_X.iloc[36612:54918],Tra_X.iloc[54918:73224],Tra_X.iloc[73224:]]
Val_5folds_ADR = [Tra_ADR[0:18306],Tra_ADR[18306:36612],Tra_ADR[36612:54918],Tra_ADR[54918:73224],Tra_ADR[73224:]]
Val_5folds_Y = [Tra_Y[0:18306],Tra_Y[18306:36612],Tra_Y[36612:54918],Tra_Y[54918:73224],Tra_Y[73224:]]

### Predict is_canceled

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

### Logistic reg with regularization

In [233]:
Accuracy = []
regularization = [0.001, 0.01, 0.05, 0.25, 0.5, 1, 10, 100, 1000]
for c in regularization:
    E_val_acc = []
    for i in range(5):
        lr = LogisticRegression(C=c)
        lr.fit(Tra_5folds_X[i] , Tra_5folds_Y[i] )
        E_val_acc.append(accuracy_score(Val_5folds_Y[i], lr.predict(Val_5folds_X[i])))
    Accuracy.append(np.mean(E_val_acc))    

C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regre

In [301]:
highestC = [regularization[i] for i in range(len(regularization)) if Accuracy[i] == max(Accuracy)][0]
print('highest C:',highestC)
print('E_val_accuracy:',max(Accuracy))

final_model = LogisticRegression(C = highestC)
final_model.fit(X_train_c, Y_train_Cancel)
print(f'Train Acc: {accuracy_score(Y_train_Cancel, final_model.predict(X_train_c))}')

highest C: 10
E_val_accuracy: 0.8228777450016389
Train Acc: 0.8244400742925817


C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [262]:
y_pred_log = final_model.predict(X_test_c)

In [86]:
Test_data = pd.read_csv("D:/test.csv")
Cancel_label =  pd.DataFrame() 
Cancel_label['arrival_date'] = Test_data['ID']
Cancel_label['logistic'] = y_pred_log

### Decision Tree Classifier

In [100]:
from sklearn.tree import DecisionTreeClassifier

Cri = ['gini','entropy']
for c in Cri:
    print('Criterion:',c)
    for j in range(15):
        k=j+1
        E_val_acc = []
        for i in range(5):
            model1=DecisionTreeClassifier(max_depth=k,criterion=c)
            model1.fit(Tra_5folds_X[i] , Tra_5folds_Y[i])
            E_val_acc.append(accuracy_score(Val_5folds_Y[i], lr.predict(Val_5folds_X[i])))
        print(f'E_val_acc for j={k}: {np.mean(E_val_acc)}')

Criterion: gini
E_val_acc for j=1: 0.8238391784114498
E_val_acc for j=2: 0.8238391784114498
E_val_acc for j=3: 0.8238391784114498
E_val_acc for j=4: 0.8238391784114498
E_val_acc for j=5: 0.8238391784114498
E_val_acc for j=6: 0.8238391784114498
E_val_acc for j=7: 0.8238391784114498
E_val_acc for j=8: 0.8238391784114498
E_val_acc for j=9: 0.8238391784114498
E_val_acc for j=10: 0.8238391784114498
E_val_acc for j=11: 0.8238391784114498
E_val_acc for j=12: 0.8238391784114498
E_val_acc for j=13: 0.8238391784114498
E_val_acc for j=14: 0.8238391784114498
E_val_acc for j=15: 0.8238391784114498
Criterion: entropy
E_val_acc for j=1: 0.8238391784114498
E_val_acc for j=2: 0.8238391784114498
E_val_acc for j=3: 0.8238391784114498
E_val_acc for j=4: 0.8238391784114498
E_val_acc for j=5: 0.8238391784114498
E_val_acc for j=6: 0.8238391784114498
E_val_acc for j=7: 0.8238391784114498
E_val_acc for j=8: 0.8238391784114498
E_val_acc for j=9: 0.8238391784114498
E_val_acc for j=10: 0.8238391784114498
E_val_ac

In [322]:
final_model = DecisionTreeClassifier(max_depth=12,criterion='entropy')
final_model.fit(X_train_c, Y_train_Cancel)
print(f'Train Acc: {accuracy_score(Y_train_Cancel, final_model.predict(X_train_c))}')

Train Acc: 0.8661531738227903


In [273]:
y_pred_dt = final_model.predict(X_test_c)
Cancel_label['decision'] = y_pred_dt

### RandomForest

In [109]:
from sklearn.ensemble import RandomForestClassifier
md1_rf = RandomForestClassifier(max_depth=50, n_estimators=1000,random_state=1)
md1_rf.fit(X_train_c, Y_train_Cancel)
y_pred = md1_rf.predict(X_train_c)
accuracy_score(Y_train_Cancel, y_pred)

0.9838304381077242

In [120]:
from sklearn.ensemble import RandomForestClassifier
# Model Selection by best E_oob_accuracy

C = [30,40,50,60,70]
E_oob_acc=[]
for i in C:
    md1_rf = RandomForestClassifier(max_depth=i, n_estimators=1000,oob_score=True,random_state=1,n_jobs=-1)
    md1_rf.fit(X_train_c, Y_train_Cancel)
    y_pred = md1_rf.predict(X_train_c)
    E_oob_acc.append(accuracy_score(Y_train_Cancel, y_pred))

In [122]:
print(E_oob_acc)

[0.9384682617720966, 0.9708510870752759, 0.9838304381077242, 0.9860264394187698, 0.9861466185949962]


In [288]:
md1_rf = RandomForestClassifier(max_depth=70, n_estimators=1000,random_state=1,n_jobs=-1)
md1_rf.fit(X_train_c, Y_train_Cancel)
y_pred = md1_rf.predict(X_train_c)
print(f'Train Acc: {accuracy_score(Y_train_Cancel, y_pred)}')

Train Acc: 0.9861466185949962


In [291]:
y_pred_rf = md1_rf.predict(X_test_c)
Cancel_label['rf'] = y_pred_rf

In [132]:
Cancel_label.to_csv(r'C:\Users\USER\Desktop\record.csv',index=False)

### Neural Network Classifier

In [136]:
from sklearn.tree import DecisionTreeClassifier

activation = ['relu']

for a in activation:
    print('Activation:',a)
    for j in [400,500]:
        E_val_acc = []
        for i in range(5):
            NN_model =  MLPClassifier(random_state=0,hidden_layer_sizes=j,activation=a)
            NN_model.fit(Tra_5folds_X[i] , Tra_5folds_Y[i])
            E_val_acc.append(accuracy_score(Val_5folds_Y[i], lr.predict(Val_5folds_X[i])))
        print(f'E_val_acc for layer={j}: {np.mean(E_val_acc)}')

Activation: relu


C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


KeyboardInterrupt: 

### Neural Network Classifier

In [59]:
from sklearn.neural_network import MLPClassifier
NN_model =  MLPClassifier(random_state=0,hidden_layer_sizes=500)
NN_model.fit(x_train, y_train)
y_pred = NN_model.predict(x_test)
print(confusion_matrix(y_test, y_pred))
accuracy_score(y_test, y_pred)

C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:573: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


[[10716  1030]
 [ 1344  5216]]


0.8703157434720856

In [62]:
NN_pred = NN_model.predict(X_test_c )
NN_pred

array([0, 1, 0, ..., 1, 0, 0], dtype=int64)

In [89]:
# Use all data
from sklearn.neural_network import MLPClassifier
NN_model =  MLPClassifier(random_state=0,hidden_layer_sizes=500)
NN_model.fit(X_train_c, Y_train_c)
y_pred = NN_model.predict(X_train_c)
print(confusion_matrix(Y_train_c, y_pred))
accuracy_score(Y_train_c, y_pred)

[[58203   568]
 [ 1011 31748]]


0.9827488255216869

In [90]:
allNN_pred = NN_model.predict(X_test_c )
allNN_pred

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

### Gradient Boosting Classifier

In [303]:
from sklearn.ensemble import GradientBoostingClassifier

gb_model = GradientBoostingClassifier(max_depth= 10)
gb_model.fit(X_train_c, Y_train_Cancel)
y_pred = gb_model.predict(X_train_c)
print(confusion_matrix(Y_train_Cancel, y_pred))
accuracy_score(Y_train_Cancel, y_pred)

[[55835  2936]
 [ 6242 26517]]


0.8997268655085764

In [304]:
gb_pred = gb_model.predict(X_test_c )
gb_pred

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

### AdaBoost Classifier

In [73]:
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=1000, random_state=0)
ada.fit(x_train, y_train)
y_pred = ada.predict(x_test)
print(confusion_matrix(y_test, y_pred))
accuracy_score(y_test, y_pred)


[[10823   923]
 [ 2341  4219]]


0.821697803998689

In [75]:
ada_pred = ada.predict(X_test_c )
ada_pred

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

In [91]:
# Use all data
ada = AdaBoostClassifier(n_estimators=1000, random_state=0)
ada.fit(X_train_c, Y_train_c)
y_pred = ada.predict(X_train_c)
print(confusion_matrix(Y_train_c, y_pred))
accuracy_score(Y_train_c, y_pred)

[[54129  4642]
 [11512 21247]]


0.8235114170217415

In [92]:
allada_pred = ada.predict(X_test_c )
allada_pred

array([0, 1, 0, ..., 0, 0, 0], dtype=int64)

### Uniform Blending for all classifier

In [77]:
# Use incomplete data
Blend_C = [(dt_pred[i]+rf_pred[i]+NN_pred[i]+gb_pred[i]+ada_pred[i]+logistic_pred[i])/6 for i in range(len(dt_pred))]
for i in range(len(Blend_C)):
    if Blend_C[i]>=0.5:
        Blend_C[i] = 1
    else: Blend_C[i]=0

In [96]:
# Use ALL data
AllBlend_C = [(alldt_pred[i]+allrf_pred[i]+allNN_pred[i]+allgb_pred[i]+allada_pred[i]+alllogistic_pred[i])/6 for i in range(len(alldt_pred))]
for i in range(len(AllBlend_C)):
    if AllBlend_C[i]>=0.5:
        AllBlend_C[i] = 1
    else: AllBlend_C[i]=0

# Prediction of ADR

### Linear Regression

In [152]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
E_val_mse = []
for i in range(5):
    reg = LinearRegression()
    reg.fit(Tra_5folds_X[i] , Tra_5folds_ADR[i] )
    print(mean_squared_error(Tra_5folds_ADR[i], reg.predict(Tra_5folds_X[i])))
    E_val_mse.append(mean_squared_error(Val_5folds_ADR[i], reg.predict(Val_5folds_X[i])))
print(E_val_mse)

1094.3492768788053
1096.1857551220705
1094.6863630872458
1094.302576562821
1100.3253535570843
[3.0827245159192695e+23, 1.1568795001828866e+23, 2.290983901326358e+23, 7.31119724055254e+22, 2.0814065750867614e+22]


In [153]:
reg = LinearRegression().fit(X_train_c, Y_train_ADR)
y_pred = reg.predict(X_train_c)
print('MSE of test:', mean_squared_error(Y_train_ADR, y_pred))

MSE of test: 1096.7804313372787


In [158]:
reg_pred = reg.predict(X_test_c )
reg_pred

array([100.125    ,  79.25     ,  71.1171875, ..., 149.1015625,
        95.0546875, 155.390625 ])

### Ridge Regression

In [164]:
from sklearn.linear_model import Ridge
Accuracy = []
regularization = [0.001, 0.01, 0.05, 0.25, 0.5, 1, 10, 100, 1000]
for c in regularization:
    E_val_mse = []
    for i in range(5):
        lridge = Ridge(alpha =c)
        lridge.fit(Tra_5folds_X[i] , Tra_5folds_ADR[i])
        E_val_mse.append(mean_squared_error(Val_5folds_ADR[i], lridge.predict(Val_5folds_X[i])))
    Accuracy.append(np.mean(E_val_mse))  
print(Accuracy)

[1104.5432258944538, 1104.537758679981, 1104.5148252807583, 1104.4253885796074, 1104.3498583311318, 1104.2556302876558, 1104.1262643862037, 1108.1575447599566, 1165.0901382645745]


In [166]:
bestC = [regularization[i] for i in range(len(regularization)) if Accuracy[i] ==min(Accuracy)][0]
print('highest C:',bestC)
print('Min MSE:',min(Accuracy))

lridge_model = Ridge(alpha = bestC )
lridge_model.fit(X_train_c, Y_train_ADR)
print(f'Train Acc: {mean_squared_error(Y_train_ADR, lridge_model.predict(X_train_c))}')


highest C: 10
Min MSE: 1104.1262643862037
Train Acc: 1097.6137835402112


In [167]:
lridge_pred = lridge_model.predict(X_test_c)
lridge_pred

array([101.06923055,  80.25408028,  71.91809973, ..., 148.22254823,
        94.23839143, 153.09713845])

### Neural Network Regressor

In [309]:
from sklearn.neural_network import MLPRegressor

NNreg_model = MLPRegressor(random_state=1, hidden_layer_sizes=200)
NNreg_model.fit(X_train_c, Y_train_ADR)
y_pred = NNreg_model.predict(X_train_c)
print(f'Test MSE: {mean_squared_error(Y_train_ADR, y_pred)}')

C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Test MSE: 382.8109145628419


In [310]:
real_NNreg_pred = NNreg_model.predict(X_test_c)
real_NNreg_pred 

array([103.50209291,  80.69683205,  84.10910136, ..., 207.99900711,
        88.38148714, 128.92922495])

### Gradient Boosting Regressor

In [107]:
from sklearn.ensemble import GradientBoostingRegressor
gbreg = GradientBoostingRegressor(random_state=0,max_depth=5)
gbreg.fit(x_train, y_train)
y_pred = gbreg.predict(x_test)
print(f'Test MSE: {mean_squared_error(y_test, y_pred)}')

Train MSE: 684.733587393979


In [119]:
gbreg_pred = gbreg.predict(X_test_c)
gbreg_pred

array([ 88.36772984,  64.22226255,  50.12183555, ..., 205.18923498,
        98.91291725, 116.07145615])

### AdaBoostRegressor

In [147]:
from sklearn.ensemble import AdaBoostRegressor
adareg = AdaBoostRegressor(random_state=0, n_estimators=200)
adareg.fit(x_train, y_train)
y_pred = adareg.predict(x_test)
print(f'Test MSE: {mean_squared_error(y_test, y_pred)}')

Test MSE: 1858.8954721972934


In [148]:
adareg_pred = adareg.predict(X_test_c)
adareg_pred

array([103.93772325,  98.5873158 ,  79.6147115 , ..., 148.92589746,
       126.25936467, 122.95984582])

In [ ]:
# Train label accuracy
def Tr_Daily_Revenue(Test_Data,Test_label, cancel_prediction, adr_prediction):
    Test_Data = Test_Data.copy() 
    Test_Data['adr']= adr_prediction
    Test_Data['is_canceled'] = cancel_prediction
    
    Test_Data['arrival_date_month'] = pd.to_datetime(Test_Data.arrival_date_month, format='%B').dt.month
    Test_Data['dateInt']=Test_Data['arrival_date_year'].astype(str)+Test_Data['arrival_date_month'].astype(str).str.zfill(2)+ Test_Data['arrival_date_day_of_month'].astype(str).str.zfill(2)
            
    Test_Data['arrive_Date'] = pd.to_datetime(Test_Data['dateInt'], format='%Y-%m-%d')
    Test_Data['total_stay'] = Test_Data['stays_in_weekend_nights'] + Test_Data['stays_in_week_nights']
    Target = Test_Data.loc[Test_Data['is_canceled'] ==0]
    
    Pre = []
    D = Test_label['arrival_date']
    for i in range(len(D)):
        Ju = Target.loc[Target['arrive_Date']==D[i]]
        Q = math.floor((np.dot(Ju['total_stay'].values,Ju['adr'].values))/10000)
        Pre.append(Q)
        
    return Pre

In [ ]:
# Test label 
def Daily_Revenue(Test_Data,Test_label, cancel_prediction, adr_prediction):
    Test_Data = Test_Data.copy() 
    Test_Data['adr']= adr_prediction
    Test_Data['is_canceled'] = cancel_prediction
    
    Test_Data['arrival_date_month'] = pd.to_datetime(Test_Data.arrival_date_month, format='%B').dt.month
    Test_Data['dateInt']=Test_Data['arrival_date_year'].astype(str)+Test_Data['arrival_date_month'].astype(str).str.zfill(2)+ Test_Data['arrival_date_day_of_month'].astype(str).str.zfill(2)
            
    Test_Data['arrive_Date'] = pd.to_datetime(Test_Data['dateInt'], format='%Y-%m-%d')
    Test_Data['total_stay'] = Test_Data['stays_in_weekend_nights'] + Test_Data['stays_in_week_nights']
    Target = Test_Data.loc[Test_Data['is_canceled'] ==0]
    
    Pre = []
    D = Test_label['arrival_date']
    for i in range(len(D)):
        Ju = Target.loc[Target['arrive_Date']==D[i]]
        Q = math.floor((np.dot(Ju['total_stay'].values,Ju['adr'].values))/10000)
        Pre.append(Q)
        
    return Pre

In [302]:
# Logistic + Ridge Reg
from sklearn.metrics import confusion_matrix
lab_pred = Tr_Daily_Revenue(X_train, Y_train, final_model.predict(X_train_c), lridge_model.predict(X_train_c))
print(confusion_matrix(Y_train['label'], lab_pred))
print(accuracy_score(Y_train['label'], lab_pred))
print(mean_absolute_error(Y_train['label'], lab_pred))

[[ 74  20   0   0   0   0   0   0   0   0]
 [  7 102  41   2   0   0   0   0   0   0]
 [  0   9 125  46   6   0   0   0   0   0]
 [  0   0   4  78  38   4   0   0   0   0]
 [  0   0   0   7  27  10   2   0   0   0]
 [  0   0   0   0   8   9   4   0   0   0]
 [  0   0   0   0   1   4   8   0   0   0]
 [  0   0   0   0   0   1   0   0   0   0]
 [  0   0   0   0   0   0   0   2   0   0]
 [  0   0   0   0   0   0   1   0   0   0]]
0.6609375
0.3671875


In [ ]:
# Test
import math 
DR = Daily_Revenue(X_test,Y_test, y_pred_log, lridge_pred)
Test_label['label'] = DR
Test_label.to_csv(r'C:\Users\USER\Desktop\UB.csv',index=False)
print(np.array(DR))

In [280]:
#Test_pred_label =  pd.DataFrame() 
Test_pred_label['arrival_date'] = Y_test['arrival_date']
Test_pred_label['log&rid'] = DR


In [300]:
# Decision + Ridge
lab_pred = Tr_Daily_Revenue(X_train, Y_train, final_model.predict(X_train_c), lridge_model.predict(X_train_c))
print(confusion_matrix(Y_train['label'], lab_pred))
print(accuracy_score(Y_train['label'], lab_pred))
print(mean_absolute_error(Y_train['label'], lab_pred))

[[ 76  18   0   0   0   0   0   0   0   0]
 [  9 107  35   1   0   0   0   0   0   0]
 [  0  20 125  38   3   0   0   0   0   0]
 [  0   0  12  84  24   4   0   0   0   0]
 [  0   0   0   8  28   7   3   0   0   0]
 [  0   0   0   1   8   8   4   0   0   0]
 [  0   0   0   0   1   4   6   2   0   0]
 [  0   0   0   0   0   0   1   0   0   0]
 [  0   0   0   0   0   0   0   0   1   1]
 [  0   0   0   0   0   0   1   0   0   0]]
0.6796875
0.34375


In [284]:
# Test
test_pred = Daily_Revenue(X_test, Y_test, y_pred_dt, lridge_pred)
Test_pred_label['dt&rid'] = test_pred

Test_label['label'] = test_pred
Test_label.to_csv(r'C:\Users\USER\Desktop\UB.csv',index=False)
#Test_label.to_csv(r'C:\Users\USER\Desktop\Test_record.csv',index=False)

[3, 2, 3, 1, 2, 2, 2, 3, 6, 5, 3, 3, 6, 4, 4, 3, 3, 1, 2, 2, 3, 3, 4, 3, 2, 1, 3, 5, 7, 3, 5, 4, 3, 4, 4, 4, 3, 4, 3, 2, 3, 2, 3, 3, 3, 2, 2, 3, 2, 2, 4, 3, 2, 4, 7, 4, 3, 5, 4, 3, 2, 4, 6, 5, 4, 5, 2, 2, 3, 3, 3, 4, 3, 2, 2, 4, 2, 3, 5, 5, 3, 3, 3, 4, 5, 4, 5, 3, 3, 4, 4, 5, 3, 5, 3, 2, 3, 3, 3, 5, 5, 4, 3, 3, 3, 6, 5, 6, 3, 2, 3, 3, 4, 4, 6, 4, 4, 4, 4, 6, 5, 5, 3, 4, 4, 3, 4, 4, 7, 3, 3, 4, 3, 5, 4, 4, 3, 4, 4, 4, 4, 5, 4, 2, 4, 4, 4, 5, 4, 4, 3, 2, 4]


In [299]:
# RF + Ridge
lab_pred = Tr_Daily_Revenue(X_train, Y_train, md1_rf.predict(X_train_c), lridge_model.predict(X_train_c))
print(confusion_matrix(Y_train['label'], lab_pred))
print(accuracy_score(Y_train['label'], lab_pred))
print(mean_absolute_error(Y_train['label'], lab_pred))

[[ 90   4   0   0   0   0   0   0   0   0]
 [  9 126  17   0   0   0   0   0   0   0]
 [  0  23 144  19   0   0   0   0   0   0]
 [  0   0  14  95  15   0   0   0   0   0]
 [  0   0   0  11  31   4   0   0   0   0]
 [  0   0   0   2  10   9   0   0   0   0]
 [  0   0   0   0   2   4   7   0   0   0]
 [  0   0   0   0   0   0   1   0   0   0]
 [  0   0   0   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   1   0   0   0]]
0.7859375
0.2234375


In [295]:
# Test
test_pred = Daily_Revenue(X_test, Y_test, y_pred_rf, lridge_pred)
Test_pred_label['rf&rid'] = test_pred

Test_label['label'] = test_pred
Test_label.to_csv(r'C:\Users\USER\Desktop\UB.csv',index=False)
print(test_pred)
Test_pred_label.to_csv(r'C:\Users\USER\Desktop\Test_record.csv',index=False)

[3, 3, 4, 1, 2, 2, 2, 4, 5, 4, 3, 2, 6, 4, 4, 4, 4, 2, 3, 2, 3, 4, 4, 3, 2, 1, 3, 7, 6, 4, 5, 4, 3, 4, 5, 5, 4, 5, 3, 2, 3, 3, 4, 4, 4, 3, 3, 4, 3, 3, 4, 4, 3, 5, 7, 4, 4, 6, 4, 3, 3, 5, 6, 6, 5, 6, 3, 2, 5, 4, 4, 6, 4, 3, 3, 5, 4, 5, 5, 5, 4, 4, 4, 4, 5, 5, 5, 3, 3, 3, 4, 7, 4, 6, 4, 3, 4, 4, 5, 6, 6, 4, 3, 4, 3, 8, 6, 8, 4, 3, 4, 4, 6, 6, 8, 5, 4, 5, 5, 8, 5, 7, 6, 6, 7, 6, 6, 6, 10, 4, 5, 5, 4, 8, 7, 7, 6, 7, 7, 6, 7, 8, 6, 4, 5, 5, 5, 7, 6, 10, 4, 3, 5]


In [305]:
# GB + ridge
lab_pred = Tr_Daily_Revenue(X_train, Y_train, gb_model.predict(X_train_c), lridge_model.predict(X_train_c))
print(confusion_matrix(Y_train['label'], lab_pred))
print(accuracy_score(Y_train['label'], lab_pred))
print(mean_absolute_error(Y_train['label'], lab_pred))

[[ 84  10   0   0   0   0   0   0   0   0]
 [  6 120  25   1   0   0   0   0   0   0]
 [  0  18 140  28   0   0   0   0   0   0]
 [  0   0  10  95  18   1   0   0   0   0]
 [  0   0   0  10  29   7   0   0   0   0]
 [  0   0   0   1   8  10   2   0   0   0]
 [  0   0   0   0   2   2   9   0   0   0]
 [  0   0   0   0   0   0   1   0   0   0]
 [  0   0   0   0   0   0   0   0   2   0]
 [  0   0   0   0   0   0   1   0   0   0]]
0.7640625
0.246875


In [306]:
# Test
test_pred = Daily_Revenue(X_test, Y_test, gb_pred, lridge_pred)
Test_pred_label['gb&rid'] = test_pred

Test_label['label'] = test_pred
Test_label.to_csv(r'C:\Users\USER\Desktop\UB.csv',index=False)
print(test_pred)
Test_pred_label.to_csv(r'C:\Users\USER\Desktop\Test_record.csv',index=False)

[3, 2, 4, 1, 2, 2, 2, 4, 5, 4, 3, 2, 5, 4, 3, 4, 4, 2, 3, 2, 3, 4, 4, 3, 2, 1, 3, 6, 6, 4, 5, 4, 3, 4, 5, 5, 3, 4, 3, 2, 3, 3, 4, 4, 3, 3, 3, 4, 3, 3, 4, 4, 3, 5, 7, 4, 3, 5, 4, 3, 3, 5, 6, 6, 4, 6, 2, 2, 5, 3, 4, 6, 4, 3, 3, 5, 4, 5, 5, 5, 3, 4, 4, 4, 5, 5, 5, 3, 3, 3, 4, 6, 3, 6, 4, 3, 3, 3, 4, 6, 5, 4, 3, 3, 3, 7, 6, 7, 4, 3, 3, 4, 5, 6, 7, 4, 4, 5, 4, 7, 5, 6, 5, 5, 7, 5, 6, 6, 9, 4, 4, 5, 4, 7, 7, 6, 5, 6, 6, 6, 6, 7, 5, 3, 5, 5, 5, 6, 5, 9, 4, 3, 5]


In [312]:
# Logis + NN
lab_pred = Tr_Daily_Revenue(X_train, Y_train, final_model.predict(X_train_c), NNreg_model.predict(X_train_c))
print(confusion_matrix(Y_train['label'], lab_pred))
print(accuracy_score(Y_train['label'], lab_pred))
print(mean_absolute_error(Y_train['label'], lab_pred))

[[ 79  15   0   0   0   0   0   0   0   0   0]
 [  1 108  42   0   1   0   0   0   0   0   0]
 [  0   1 135  47   3   0   0   0   0   0   0]
 [  0   0   2  74  47   1   0   0   0   0   0]
 [  0   0   0   2  29  12   3   0   0   0   0]
 [  0   0   0   0   4   7   9   1   0   0   0]
 [  0   0   0   0   0   4   5   4   0   0   0]
 [  0   0   0   0   0   0   1   0   0   0   0]
 [  0   0   0   0   0   0   1   0   1   0   0]
 [  0   0   0   0   0   0   0   0   0   0   1]
 [  0   0   0   0   0   0   0   0   0   0   0]]
0.684375
0.3328125


In [314]:
# Test
test_pred = Daily_Revenue(X_test, Y_test, y_pred_log , real_NNreg_pred )
Test_pred_label['log&NN'] = test_pred

Test_label['label'] = test_pred
Test_label.to_csv(r'C:\Users\USER\Desktop\UB.csv',index=False)
print(test_pred)
Test_pred_label.to_csv(r'C:\Users\USER\Desktop\Test_record.csv',index=False)

[3, 2, 3, 1, 2, 2, 2, 3, 5, 4, 3, 3, 5, 4, 4, 3, 3, 1, 2, 2, 3, 3, 4, 3, 2, 1, 3, 6, 6, 3, 4, 3, 2, 3, 4, 4, 3, 4, 2, 1, 3, 2, 3, 3, 3, 2, 2, 3, 2, 2, 4, 3, 2, 4, 5, 3, 2, 5, 3, 2, 2, 4, 6, 4, 4, 5, 2, 2, 4, 3, 3, 4, 4, 2, 2, 4, 3, 3, 5, 5, 3, 4, 4, 3, 5, 5, 4, 3, 3, 4, 4, 6, 3, 5, 4, 3, 4, 3, 4, 5, 5, 4, 3, 4, 4, 7, 5, 7, 5, 2, 4, 4, 4, 5, 6, 4, 4, 4, 4, 6, 5, 5, 4, 4, 5, 4, 5, 4, 8, 3, 3, 4, 5, 6, 4, 5, 3, 4, 5, 4, 4, 6, 6, 3, 5, 5, 4, 5, 4, 5, 3, 3, 4]


In [323]:
# Dec + NN
lab_pred = Tr_Daily_Revenue(X_train, Y_train, final_model.predict(X_train_c), NNreg_model.predict(X_train_c))
print(confusion_matrix(Y_train['label'], lab_pred))
print(accuracy_score(Y_train['label'], lab_pred))
print(mean_absolute_error(Y_train['label'], lab_pred))

[[ 78  16   0   0   0   0   0   0   0   0   0]
 [  4 116  32   0   0   0   0   0   0   0   0]
 [  0   5 141  40   0   0   0   0   0   0   0]
 [  0   0   2  82  38   2   0   0   0   0   0]
 [  0   0   0   3  31  12   0   0   0   0   0]
 [  0   0   0   0   3  10   8   0   0   0   0]
 [  0   0   0   0   0   3   6   3   1   0   0]
 [  0   0   0   0   0   0   0   1   0   0   0]
 [  0   0   0   0   0   0   0   0   1   0   1]
 [  0   0   0   0   0   0   0   0   0   0   1]
 [  0   0   0   0   0   0   0   0   0   0   0]]
0.728125
0.278125


In [325]:
# Test
test_pred = Daily_Revenue(X_test, Y_test, y_pred_dt , real_NNreg_pred )
Test_pred_label['dt&NN'] = test_pred

Test_label['label'] = test_pred
Test_label.to_csv(r'C:\Users\USER\Desktop\UB.csv',index=False)
print(test_pred)
Test_pred_label.to_csv(r'C:\Users\USER\Desktop\Test_record.csv',index=False)

[3, 2, 3, 1, 2, 2, 2, 4, 4, 3, 3, 2, 4, 4, 3, 3, 4, 1, 2, 2, 4, 3, 4, 3, 2, 1, 3, 6, 5, 3, 5, 3, 2, 3, 4, 5, 4, 4, 2, 2, 3, 3, 4, 3, 3, 2, 2, 3, 3, 3, 4, 4, 2, 5, 5, 3, 2, 5, 3, 2, 2, 5, 5, 5, 4, 5, 2, 2, 5, 3, 4, 7, 5, 3, 3, 6, 4, 5, 5, 5, 3, 4, 5, 4, 6, 5, 4, 3, 3, 3, 4, 8, 4, 6, 5, 3, 5, 4, 4, 6, 6, 5, 3, 5, 4, 9, 6, 8, 5, 3, 4, 5, 6, 7, 8, 4, 4, 5, 5, 7, 5, 6, 6, 5, 7, 7, 7, 7, 10, 4, 4, 5, 5, 8, 8, 7, 4, 6, 9, 6, 6, 8, 7, 4, 6, 6, 5, 7, 6, 9, 3, 3, 5]


In [320]:
# RF + NN
lab_pred = Tr_Daily_Revenue(X_train, Y_train, md1_rf.predict(X_train_c),  NNreg_model.predict(X_train_c) )
print(confusion_matrix(Y_train['label'], lab_pred))
print(accuracy_score(Y_train['label'], lab_pred))
print(mean_absolute_error(Y_train['label'], lab_pred))

[[ 90   4   0   0   0   0   0   0   0   0]
 [  0 140  12   0   0   0   0   0   0   0]
 [  0   2 172  12   0   0   0   0   0   0]
 [  0   0   5 110   9   0   0   0   0   0]
 [  0   0   0   1  41   4   0   0   0   0]
 [  0   0   0   0   2  15   4   0   0   0]
 [  0   0   0   0   0   2   9   2   0   0]
 [  0   0   0   0   0   0   0   1   0   0]
 [  0   0   0   0   0   0   0   0   2   0]
 [  0   0   0   0   0   0   0   0   0   1]]
0.9078125
0.0921875


In [321]:
# Test
test_pred = Daily_Revenue(X_test, Y_test, y_pred_rf , real_NNreg_pred )
Test_pred_label['rf&NN'] = test_pred

Test_label['label'] = test_pred
Test_label.to_csv(r'C:\Users\USER\Desktop\UB.csv',index=False)
print(test_pred)
Test_pred_label.to_csv(r'C:\Users\USER\Desktop\Test_record.csv',index=False)

[3, 2, 4, 1, 3, 2, 2, 4, 4, 3, 3, 2, 5, 4, 4, 4, 3, 2, 2, 2, 4, 4, 4, 4, 2, 1, 3, 7, 5, 3, 5, 4, 3, 3, 5, 5, 4, 5, 2, 2, 3, 3, 4, 4, 3, 2, 2, 3, 3, 3, 4, 4, 3, 5, 6, 3, 3, 6, 3, 2, 2, 5, 5, 5, 4, 5, 3, 3, 5, 4, 5, 7, 5, 3, 4, 6, 5, 6, 6, 6, 4, 4, 5, 4, 5, 5, 5, 3, 3, 3, 4, 8, 5, 6, 6, 4, 5, 5, 5, 6, 6, 5, 3, 5, 4, 9, 7, 9, 5, 3, 4, 5, 6, 7, 9, 5, 4, 5, 6, 8, 5, 7, 7, 6, 8, 7, 7, 7, 11, 5, 5, 6, 5, 9, 8, 8, 6, 8, 9, 7, 8, 8, 8, 5, 6, 6, 6, 8, 6, 10, 4, 3, 5]


In [326]:
# GB+NN
lab_pred = Tr_Daily_Revenue(X_train, Y_train,  gb_model.predict(X_train_c),  NNreg_model.predict(X_train_c) )
print(confusion_matrix(Y_train['label'], lab_pred))
print(accuracy_score(Y_train['label'], lab_pred))
print(mean_absolute_error(Y_train['label'], lab_pred))

[[ 84  10   0   0   0   0   0   0   0   0   0]
 [  3 125  24   0   0   0   0   0   0   0   0]
 [  0   5 157  24   0   0   0   0   0   0   0]
 [  0   0   4  97  23   0   0   0   0   0   0]
 [  0   0   0   3  36   7   0   0   0   0   0]
 [  0   0   0   0   4  12   4   1   0   0   0]
 [  0   0   0   0   0   1   9   3   0   0   0]
 [  0   0   0   0   0   0   0   0   1   0   0]
 [  0   0   0   0   0   0   0   0   1   1   0]
 [  0   0   0   0   0   0   0   0   0   0   1]
 [  0   0   0   0   0   0   0   0   0   0   0]]
0.8140625
0.1875


In [328]:
# Test
test_pred = Daily_Revenue(X_test, Y_test, gb_pred , real_NNreg_pred )
Test_pred_label['gb&NN'] = test_pred

Test_label['label'] = test_pred
Test_label.to_csv(r'C:\Users\USER\Desktop\UB.csv',index=False)
print(test_pred)
Test_pred_label.to_csv(r'C:\Users\USER\Desktop\Test_record.csv',index=False)

[3, 2, 3, 1, 3, 2, 2, 4, 4, 3, 3, 2, 4, 3, 3, 4, 3, 2, 3, 2, 4, 3, 4, 3, 2, 1, 3, 6, 5, 3, 5, 3, 2, 3, 4, 5, 3, 4, 2, 2, 3, 3, 4, 4, 3, 2, 2, 4, 3, 3, 4, 4, 3, 5, 5, 3, 3, 5, 3, 2, 2, 5, 5, 5, 4, 5, 2, 2, 5, 3, 4, 7, 5, 3, 3, 6, 4, 5, 5, 5, 4, 4, 4, 4, 4, 5, 4, 3, 3, 3, 4, 7, 4, 6, 5, 3, 4, 3, 4, 6, 6, 5, 3, 5, 4, 8, 7, 8, 5, 3, 4, 5, 6, 6, 8, 4, 4, 5, 5, 7, 5, 6, 6, 5, 7, 6, 7, 7, 10, 4, 5, 5, 5, 8, 7, 7, 5, 7, 7, 7, 6, 7, 7, 4, 5, 6, 5, 7, 6, 9, 4, 3, 5]


### Uniform Blending

In [350]:
Test_pred_label.iloc[:,1:].to_numpy().shape

(153, 8)

In [354]:
list(Test_pred_label.iloc[:,1:].to_numpy()[i]) 

[2, 2, 2, 2, 2, 3, 2, 3]

In [356]:
def most_frequent(List): 
    return max(set(List), key = List.count) 
  
UB = [most_frequent(list(Test_pred_label.iloc[:,1:].to_numpy()[i])) for i in range(153)]
print(UB)

Test_label['label'] = UB
Test_label.to_csv(r'C:\Users\USER\Desktop\UB.csv',index=False)

[3, 2, 3, 1, 2, 2, 2, 4, 4, 4, 3, 2, 5, 4, 3, 4, 3, 1, 2, 2, 3, 3, 4, 3, 2, 1, 3, 6, 6, 3, 5, 3, 3, 3, 4, 5, 3, 4, 2, 2, 3, 3, 4, 4, 3, 2, 2, 3, 3, 3, 4, 4, 2, 5, 7, 3, 3, 5, 3, 2, 2, 5, 5, 5, 4, 5, 2, 2, 5, 3, 4, 6, 4, 3, 3, 5, 4, 5, 5, 5, 3, 4, 4, 4, 5, 5, 4, 3, 3, 3, 4, 7, 4, 6, 4, 3, 4, 3, 4, 6, 5, 4, 3, 3, 3, 8, 6, 8, 5, 3, 4, 4, 6, 6, 8, 4, 4, 5, 4, 7, 5, 6, 6, 5, 7, 5, 6, 6, 10, 4, 3, 5, 5, 8, 7, 7, 3, 6, 6, 6, 6, 8, 6, 3, 5, 5, 5, 7, 6, 9, 3, 3, 5]
